# Загрузить датасет

In [192]:
# Добавляем импорты

from gensim.models import FastText
import pandas as pd
import numpy as np
import pymorphy2
import nltk  # Natural Language Toolkit - для загрузки стоп слов русского языка
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm  # прогресбарbincount

from sklearn.model_selection import train_test_split  # рабиваем данные на traint и test

### Создаем утилиты для работы с датасетом

In [3]:
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except Exception as e:
        return ""

In [4]:
def process_data(data, all_stop_words) -> []:
    word_tokenizer = nltk.WordPunctTokenizer()

    texts = []
    targets = []

    # инициализируем лемматизатор
    morph = pymorphy2.MorphAnalyzer()

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):
        text_lower = words_only(item)  # оставим только слова
        tokens = word_tokenizer.tokenize(texкраснооармеецt_lower)  # разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова, а так же лемитизируем  слова
        tokens = [morph.parse(word)[0].normal_form for word in tokens if
                  (word not in all_stop_words and not word.isnumeric())]

        # texts.append(tokens) # добавляем в предобработанный список
        texts.append(' '.join(tokens))

    return texts

In [5]:
def read_prepared_datasets(file_name) -> []:
    result = []
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file.readlines():
            # result.append(line.replace('\n', ''))
            result.append(line)
    return result

In [6]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avetall87/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
prepared_dataset_file_name = 'dataset/prepared_dataset.txt'

# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
add_stop_words = ['ао', 'оао', 'ооо']
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь',
            'декабрь', ]
all_stop_words = stop_words + add_stop_words + months

df = pd.read_csv('dataset/dataset.csv', sep=',', engine='python')

y = df['title'].tolist()

In [8]:
texts = read_prepared_datasets(prepared_dataset_file_name)
len(texts)
texts[0:3]

['подвижный сбор год днёвка деревня катеринный\n',
 'герой советский союз лётчик медноног скульптор м аникушин генеральный консул народный республика болгария ленинград вангел петрович трынок слева направо мастерская скульптор\n',
 'дважды герой соц труд смирнов избирательный участок время выборы местный совет народный судья\n']

# Создаем модель FastText и обучаем ее

In [9]:
prepared_vocabulary = []

for words in texts:
  splitted_data = words.split(' ')
  prepared_vocabulary.append(splitted_data)

In [10]:
model = FastText(prepared_vocabulary,min_count=1, vector_size=100, workers=4)

In [11]:
pretrained_vectors = model.wv

In [190]:
#TF-IDF # needs a list of lists for words and docs along with a fasttext 'model'
text = []
for i in tqdm(prepared_vocabulary):
  string = ' '.join(i)
  text.append(string)
tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(text)
tfidf_feat = list(tf_idf_vect.get_feature_names_out())

100%|██████████| 116000/116000 [00:00<00:00, 949257.85it/s]


In [61]:
tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list

row=0;
errors=0
for sent in tqdm(texts): # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent.split(' '): # for each word in a review/sentence
        word = word.replace('\n', '') # костыль
        try:
            vec = model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf [row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except Exception as e:
            # print(e.__traceback__)
            errors =+1
            pass
    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))

    tfidf_sent_vectors.append(sent_vec)
    row += 1
print('errors noted: '+str(errors))

 82%|████████▏ | 95200/116000 [03:36<00:46, 444.84it/s]/tmp/ipykernel_431687/1614986564.py:20: RuntimeWarning: invalid value encountered in divide
  sent_vec /= weight_sum
100%|██████████| 116000/116000 [04:25<00:00, 437.63it/s]

errors noted: 1


In [62]:
df['FT_tfidf'] = tfidf_sent_vectors

In [63]:
df.head(10)

,title,text,FT_tfidf
0,6_KffdPhotoDoc_65243,Подвижной сбор 1901 года.Дневка у деревни Кате...,"[0.4274079408952849, 1.1096172831857587, -0.32..."
1,6_KffdPhotoDoc_941,"Герой Советского Союза, летчик В.А.Медноногов,...","[-0.046664259157152686, -0.15360252287640344, ..."
2,6_KffdPhotoDoc_6002,Дважды Герой Соц.Труда В.А.Смирнов на избирате...,"[-0.600270424404498, -0.18900403293787302, -0...."
3,6_KffdPhotoDoc_65096,Вид сверху на казармы полка.,"[-0.19469520797679657, 2.2874588868352457, 0.6..."
4,6_KffdPhotoDoc_95843,Генерал казачьих войск Холмский (сидит 2-ой сл...,"[-0.347157855308892, 0.9242511216400016, 1.307..."
5,6_KffdPhotoDoc_66745,Секции армокаркаса у подножия бокового корпуса.,"[-0.15875997485742965, 1.113579837411286, 0.31..."
6,6_KffdPhotoDoc_58622,Кавалерийские части на параде войск проходят м...,"[-0.7148876100601762, 0.4594124530887603, 1.34..."
7,6_KffdPhotoDoc_71835,Пассажирский поезд прибыл на Финляндский вокзал.,"[-1.0832420299290293, 0.40646691571440474, 1.0..."
8,6_KffdPhotoDoc_112688,"Хирург, профессор, орденоносец И.П.Виноградов ...","[-0.5762575419288866, 0.4494334174335709, 0.68..."
9,6_KffdPhotoDoc_18624,Скульптор В.Б. Пинчук за работой над скульптур...,"[0.21281676094631377, 0.5626009489365782, 0.52..."


In [251]:
# Сохраняем данные в файл
df.to_csv(path_or_buf='dataset/result.csv', encoding='utf-8', index_label=['title', 'FT_tfidf'])

In [235]:
def get_vector_by_text(text) -> []:
    tfidf_sent_vectors = []

    tokenizer = TfidfVectorizer(stop_words=None)
    tokenized_data = tokenizer.fit_transform(text)
    feature_names = list(tokenizer.get_feature_names_out())

    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in text:
        vector = model.wv[word]
        # obtain the tf_idfidf of a word in a sentence/review
        tfidf = tokenized_data [0, feature_names.index(word)]
        sent_vec += (vector * tfidf)
        weight_sum += tfidf
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    return tfidf_sent_vectors
    


In [256]:
from sklearn.metrics.pairwise import cosine_similarity

search_query = 'сельскохозяйственные угодия Ленинграда'.lower().split(' ')

query = get_vector_by_text(process_data(search_query, all_stop_words))

query = np.array(list(query))
query = np.nan_to_num(query)

vectors = np.array(list(df.FT_tfidf.values))
vectors = np.nan_to_num(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(3).items():
  print({'weight':str(j), 'entity_chiper':df.title.iloc[i], 'raw text':df.text.iloc[i]})

100%|██████████| 3/3 [00:00<00:00, 5291.38it/s]

['сельскохозяйственный', 'угодия', 'ленинград']
['ленинград', 'сельскохозяйственный', 'угодия']
{'weight': '0.9248039822132548', 'entity_chiper': '6_KffdPhotoDoc_79515', 'raw text': 'Посетители областной сельскохозяйственной выставки осматривают сельскохозяйственную технику( панорама из 2-х снимков).'}
{'weight': '0.9215985248897494', 'entity_chiper': '6_KffdPhotoDoc_79572', 'raw text': 'Ленинградские ученые - участники Всесоюзной сельскохозяйственной выставки.Слева направо: директор Всесоюзного научно-исследовательского  института сельскохозяйственной микробиологии , действительный член Всесоюзной академии сельскохозяйственных наук им.В.И.Ленина И.И.Самойлов , заведующая лабораторией переработки сельскохозяйственной продукции к.б.н. М.М.Макарова и заведующий лабораторией биологических удобрений к.б.н.Л.М.Доросинский.'}
{'weight': '0.9205426376187464', 'entity_chiper': '6_KffdPhotoDoc_79563', 'raw text': 'Ленинградские ученые - участники Всесоюзной сельскохозяйственной выставки (слева 